In [2]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
using Printf
using JSON
using CSV
using DataFrames
using ColorSchemes
using LaTeXStrings

In [3]:
include("../src/SailorMoon.jl")   # relative path to main file of module
param3b = SailorMoon.dynamics_parameters()
paramMulti = SailorMoon.multi_shoot_parameters(param3b)
gr()
fs = 16


┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector}}, Tuple{ODEProblem, Real, Union{Nothing, Vector}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs\Docs.jl:240


16

In [4]:
function plot_circle(radius, x, y, n=100)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

plot_circle (generic function with 2 methods)

In [5]:
moon     = SailorMoon.plot_circle((1-param3b.mu2)*param3b.lstar, 0.0, 0.0)
earth    = SailorMoon.plot_circle(param3b.mu2*param3b.lstar, 0.0, 0.0)
earth_ub = SailorMoon.plot_circle(param3b.mu2*param3b.lstar+6375,0.0, 0.0)
earth_lb = SailorMoon.plot_circle(param3b.mu2*param3b.lstar-6375, 0.0, 0.0)
moon_soi_outer = plot_circle((1-param3b.mu2)*param3b.lstar+66000, 0.0, 0.0);

In [6]:
function interpolatecolor(cs,value, vmin, vmax)
    cs = cgrad(cs)
    color = cs[(value - vmin) / (vmax - vmin)]
    return color
end

interpolatecolor (generic function with 1 method)

In [24]:
### PARAMETERS ###################################
# csv file to load the initial solution
filename = "../run/data/diffcorr_0619_NoThrust.csv"
# dv_dir function corresponding to the csv file 
dir_func = SailorMoon.dv_no_thrust

n_arc = 5

cs = :lightrainbow 
##################################################

if dir_func == SailorMoon.dv_no_thrust
    τ_ig = 0.0
else 
    τ_ig = 1.0
end

# load initial guess
df = CSV.read(filename, DataFrame; header=0);

arcs_color = Dict(
    "xlr_bck_arc" => :blue, 
    "xlr_fwd_arc" => :blue,
    "mid_bck_arc" => :blue, 
    "mid_fwd_arc" => :blue, 
    "lpo_arc" => :blue
)




p_ = plot(
    size=(700,500), frame_style=:box, grid=0.4, legend=false, 
    xlabel=L"time, s", ylabel=L"value",
    colorbar_titlefontsize=fs,
    fontsize=fs, xlabelfontsize=fs, ylabelfontsize=fs, xtickfontsize=fs-1, ytickfontsize=fs-1, legendfontsize=fs-1
);

In [85]:

tofs= [row[1] for row in eachrow( df ) ] 
tof_max = maximum(tofs)
tof_min = minimum(tofs)

tof_max_days = tof_max * param3b.tstar / (24*60*60)
tof_min_days = tof_min * param3b.tstar / (24*60*60)

colors = [interpolatecolor(cs, row[1], tof_min, tof_max) for row in eachrow(df)]

global u = []
global t = []

# for (i, row) in enumerate( eachrow( df ) ) 
    
row = df[1,:]

    x = collect(values(row[4:end]))
    println(x)
    
#     color = colors[i]
    res, sol_param_list, sols_ballistic, tofs = SailorMoon.multishoot_trajectory2(x, dir_func, paramMulti, true, false)
        
    # ballistic legs
    for sol_ballistic in sols_ballistic

        u = sol_ballistic.u[:]
        t = sol_ballistic.t[:]
        
    end
    
    for j = 1:Int(floor(length(sol_param_list)/paramMulti.n_arc))
        
        for k = 1:paramMulti.n_arc
                        
            if mod(j,2) == 1
                # backward propagation
                sol, _, name = sol_param_list[length(sol_param_list) - j*paramMulti.n_arc + k]
                u = vcat(u, sol.u[:])
                t_append = sol.t[:] .+ t[end]
                t = vcat(t, t_append)
                
            else
                # forward propagation
                sol, _, name = sol_param_list[length(sol_param_list)-(j-1)*paramMulti.n_arc - k + 1]
                u_append = sol.u[end:-1:1, :]
                t_append = sol.t[end:-1:1] .+ t[end]
                u = vcat(u, u_append)
                t = vcat(t, t_append)
            end

        end
        
    end
    
# end
u = Array(u)
u = hcat(u...)[:,:]

println(u)
println(t)

[389.36091764741457, -0.8206612956209202, -5.324975193310116e-24, 0.7945861687734361, -0.9589480395033531, -1.0853514365490703e-23, 1.0000000000512108, 0.5105915550195769, 4.183543066795808, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 390.22071212904024, -0.010239027316855255, 8.048849914778464e-23, -0.10454589493687384, -0.03650100030269213, 2.716714128480083e-23, 1.0000000000512108, 4.69583653293579, 6.9769147230238024, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.491696391374374e-8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00251131061131133, -0.0004131724112137599, 1.0, 6.9769688693688305, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


[387.6985533450582 387.6985502947907 387.69855290483105 387.6985650356647 387.69859516899123 387.69864593718717 387.69872631420145 387.69884015456796 387.69899372422975 387.69919125484824 387.6994366405469 387.69973325743666 387.70008254436266 387.7004880268616 387.7009508474711 387.7014711558098 387.7020523171476 387.7026936994641 387.70339691074645 387.70416201019043 387.70499107940003 387.70588434979607 387.70684294293795 387.70786730903075 387.7089597055742 387.71012029540924 387.71135135720397 387.712654069443 387.71402968957034 387.715479030642 387.7170045251783 387.7186081509088 387.72029225252123 387.72205688685125 387.72390678551244 387.72584301453935 387.72786826199246 387.72998417273976 387.73219633690337 387.7345056589709 387.7369172687063 387.7394320316423 387.74205573298923 387.7447919249773 387.7476454855355 387.7506219750588 387.7537252137076 387.7569625300549 387.76033722276225 387.7638614123646 387.76753557042554 387.77137533339396 387.77538465054243 387.7795754098559

 388.8019063154428 388.8015629988532 388.80122550284756 388.800893704878 388.8005677184672 388.8002472125178 388.79993229329665 388.7996229127136 388.7993186314878 388.7990197656856 388.7987263000032 388.79843785326517 388.7981544080247 388.7978762972746 388.79760265649236 388.7973342410268 388.7970704479107 388.7968114766944 388.79655709761545 388.79630736970506 388.7960623954765 388.79582189777955 388.7955857081521 388.7953537230068 388.79512619953476 388.794902903908 388.79468375960334 388.7944687560971 388.79425792741273 388.79405099419137 388.7938480793169 388.7936489371256 388.7934537459073 388.7932622174914 388.79307455421184 388.7928903971385 388.79270998803685 388.79253306643875 388.79235977295144 388.7921900354205 388.792023477655 388.791860275272 388.7917005759239 388.79154406840917 388.79139082308285 388.79124059392166 388.79109366996346 388.7909498631578 388.79080899951174 388.79067109192675 388.79053613534927 388.7904041345159 388.7902750701429 388.79014872865787 388.7900

 0.3112477050972325 0.3020339523397125 0.2921740753883589 0.2817567757059725 0.27086394682184334 0.2595600530251187 0.24790657809183614 0.2359562917958614 0.2237480430252195 0.21131882269648447 0.198696547874092 0.18590438065773107 0.17296106690687074 0.15988125130532457 0.1466771066152501 0.13336217416793889 0.11993854763748463 0.10641392031930438 0.09279407994458959 0.07908148974352086 0.06528015858024015 0.051391135962755935 0.03741554883527482 0.023356624202019716 0.009211039426520485 -0.005015897749930083 -0.01932718399223141 -0.033723831979281305 -0.04820415236212527 -0.06276860396019317 -0.0774192000164545 -0.09215472964587956 -0.10697636991497911 -0.1218832238537717 -0.13687717900359128 -0.1519568289479156 -0.1671249589202385 -0.18237748876891194 -0.1977194177954118 -0.2131478625038613 -0.22866449137219133 -0.24426976783634988 -0.25996617959076407 -0.27574973504670014 -0.2916223647160656 -0.3075856149208352 -0.32363954511066867 -0.3397823984853143 -0.3560141121067333 -0.3723380

 -1.1073313921974817 -1.0973286691029382 -1.0874236500036878 -1.0776141486959128 -1.0679000033961905 -1.0582800657532239 -1.0487538799067002 -1.0393211425345936 -1.0299806933387703 -1.0207325154998377 -1.0115764459304086 -1.0025123029173804 -0.9935402620146613 -0.9846610118302537 -0.9758752114062247 -0.9671839214060619 -0.9585896456615923 -0.9500941048944402 -0.9417004799884763 -0.9334142234190123 -0.9252393027590283 -0.9171837964757095 -0.9092564157631815 -0.901467938179545 -0.8938325248314809 -0.8863680618356787 -0.8790949953572742 -0.8720393478203954 -0.8652318547328829 -0.8587066461978484 -0.8525120164879252 -0.8466850261051998 -0.8412957082625183 -0.8363672800625743 -0.8320117550979 -0.828170462971056 -0.8250673614975917 -0.8223934078695916 -0.8206612956209202 -0.8206612956209202 -0.8189480593179007 -0.8162163886368404 -0.8131128054957893 -0.8092529759125511 -0.8049158953478481 -0.8000211089273866 -0.7947130571180752 -0.789017534009865 -0.7830201155013861 -0.7767700174278341 -0.77

...

 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.000000000

 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.0000000000512108 1.000000000

[0.0, -0.0017272654398015593, -0.00442314537564386, -0.007208060822282914, -0.01083135880489038, -0.01474450196678579, -0.019177816066647018, -0.023957068947447185, -0.02908695751123055, -0.034507060451602345, -0.04017688800614756, -0.0460628257413692, -0.0521127480653133, -0.058325674397073896, -0.06466898594012307, -0.07110967257977197, -0.07766037011145918, -0.08428985419192628, -0.0909966364820819, -0.09776640546689645, -0.1046047524157334, -0.11150208375431853, -0.11845736809399601, -0.125465399334766, -0.13253352211641398, -0.1396551420343275, -0.14683704649873539, -0.15407886844705135, -0.16138078251798582, -0.16874079309609794, -0.17616502017693988, -0.18365660851740412, -0.1912199095710148, -0.19884899290575161, -0.2065579711929162, -0.2143445973248966, -0.22221318425626735, -0.23016369600380973, -0.23821029179861888, -0.24634920177012082, -0.2545913933537691, -0.2629326852337459, -0.2713850581110844, -0.2799523375997594, -0.2886419813833168, -0.29746271761343657, -0.306417476

, -11.142106871070888, -11.157790154530563, -11.173231624405927, -11.188437172014092, -11.20340828411921, -11.218150610736428, -11.232666443308393, -11.246959300996306, -11.261033185340388, -11.274890622499857, -11.288535755328569, -11.30197169638684, -11.315201112641583, -11.328228047740636, -11.341054110782018, -11.353684391091065, -11.366119301690103, -11.378363373337812, -11.390418561773995, -11.402288025111515, -11.413974035112766, -11.425478658699356, -11.436804984059487, -11.447954686038607, -11.458929607707233, -11.469731603592077, -11.480362094952488, -11.49082189897723, -11.501111879695824, -11.51123244595093, -11.521182362082913, -11.530961350460165, -11.540567504390667, -11.549996414138754, -11.559245476460273, -11.568307566462863, -11.577175044627078, -11.585838180242137, -11.59428375463273, -11.602494778396258, -11.610451856925785, -11.618129967593163, -11.625499517000717, -11.632527838793411, -11.639167701821991, -11.645384607067815, -11.651109538377254, -11.656323773017

, -12.18256781099928, -12.182592441745053, -12.182616722874908, -12.182641208437515, -12.182665221855704, -12.182689328011303, -12.182713318368087, -12.182737343254084, -12.182761016414746, -12.182784944762187, -12.182808527320583, -12.18283228114729, -12.182856106671341, -12.182879618806137, -12.18290312875241, -12.182926440842502, -12.182949669246623, -12.182972873054497, -12.18299622637565, -12.183019434120045, -12.183042544914318, -12.183065529570968, -12.183088661668885, -12.183111852258808, -12.183134968266337, -12.183158007260628, -12.183180913012404, -12.183204033980743, -12.183227107663871, -12.1832499191144, -12.18327293491612, -12.183295905621335, -12.183318950036659, -12.183341949618658, -12.183364858693734, -12.18338795261415, -12.183396957324197]


In [84]:
scatter!(p_, t[:], u[1,:])


attempt to save state beyond implementation limit


BoundsError: BoundsError: attempt to access 7-element Vector{Float64} at index [1:1368]

In [82]:
u[1,:]

1368-element Vector{Float64}:
 387.6985533450582
 387.6985502947907
 387.69855290483105
 387.6985650356647
 387.69859516899123
 387.69864593718717
 387.69872631420145
 387.69884015456796
 387.69899372422975
 387.69919125484824
 387.6994366405469
 387.69973325743666
 387.70008254436266
   ⋮
 388.79462803424394
 388.7947689147667
 388.79491245491664
 388.79505703015224
 388.7952012572775
 388.79534807106234
 388.79549588297755
 388.7956454485696
 388.7957959890469
 388.79594718212314
 388.79610083997676
 388.79616108960073